# get_investigators
This notebook creates a list of grant investigators and their role for a specific grant.

Roles are:
* **isPI**: PI or co-PI for the specific grant based on NIH Reporter
* **contactPi**: Contact PI for for the specific grant based on NIH Reporter
* **depositor**: Investigator registered with dbGaP for the specific grant based on dbGaP

In [1]:
import os
import pandas as pd
import grant_query
import publication_query
import utils
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#pd.set_option('display.max_colwidth', None)

In [2]:
KG_PATH = "../kg/data"
DERIVED_DATA_PATH = "../derived_data"

In [3]:
START_FISCAL_YEAR = 2021 # starts Oct. 1, 2020

In [4]:
projects = pd.read_csv(os.path.join(DERIVED_DATA_PATH, "radx-projects.csv"))
projects.fillna("", inplace=True)
projects.query("researchInitiative == 'RADx-rad'", inplace=True)

In [5]:
publication_query.expand_name_column(projects, "studyInvestigator")
projects["studyInvestigator"] = projects.apply(lambda x: x["name"] if x["dbgapAccession"].startswith("phs") else "", axis=1)
projects.drop("name", axis=1, inplace=True)

In [6]:
print(f"Number of projects {projects.shape[0]}")
projects

Number of projects 50


,researchInitiative,dbgapAccession,projectNum,coreProjectNum,projectSerialNum,studyInvestigator,studyTitle,subProject,fullName,firstName,middleName,lastName
0,RADx-rad,phs002744.v1.p1,U01AA029345,U01AA029345,AA029345,Salaita K,Rapid Acceleration of Diagnostics - Radical (R...,Automatic Detection & Tracing,Khalid Salaita,Khalid,,Salaita
12,RADx-rad,phs002778.v1.p1,1U18TR003793-01,U18TR003793,TR003793,Stott S,Rapid Acceleration of Diagnostics - Radical (R...,Exosome,Shannon Stott,Shannon,,Stott
26,RADx-rad,phs002604.v1.p1,1U01DA053949-01,U01DA053949,DA053949,Uhlemann A,Rapid Acceleration of Diagnostics - RADICAL (R...,Wastewater,Anne-Catrin Uhlemann,Anne-Catrin,,Uhlemann
27,RADx-rad,phs002583.v1.p1,5U01HL150852-02,U01HL150852,HL150852,DeMauro EP,Rapid Acceleration of Diagnostics - Radical (...,Novel Biosensing and VOC,Edward P DeMauro,Edward,P,DeMauro
28,RADx-rad,phs002603.v1.p1,3R33HD105594-03S1,R33HD105594,HD105594,Odom-John A,Rapid Acceleration of Diagnostics - Radical (...,PreVAIL kIds,Audrey Odom-John,Audrey,,Odom-John
29,RADx-rad,phs002642.v1.p1,R01DC016112,R01DC016112,DC016112,Travers S,Rapid Acceleration of Diagnostics - Radical (R...,\nChemosensory Testing,Susan Travers,Susan,,Travers
30,RADx-rad,phs002702.v1.p1,1R44DE030842-01,R44DE030842,DE030842,Ly J,Rapid Acceleration of Diagnostics - Radical (R...,Novel Biosensing and VOC,Jeffrey Ly,Jeffrey,,Ly
31,RADx-rad,phs002685.v1.p1,1R44DE030852-01,R44DE030852,DE030852,Yao X,Rapid Acceleration of Diagnostics - Radical (R...,Novel Biosensing and VOC,Xiaohu Yao,Xiaohu,,Yao
32,RADx-rad,phs003124.v1.p1,1U18TR003780-01,U18TR003780,TR003780,Das S,Rapid Acceleration of Diagnostics - Radical (R...,Exosome,Samarjit Das,Samarjit,,Das
33,RADx-rad,phs002729.v1.p1,1U01DA053899-01,U01DA053899,DA053899,Noble R,Rapid Acceleration of Diagnostics - Radical (R...,Wastewater,Rachel Noble,Rachel,,Noble


In [7]:
project_num = list(projects["coreProjectNum"].unique())

In [8]:
grant_pis = grant_query.get_principal_investigators(list(project_num))
# keep only the entries for the most recent fiscal year
grant_pis["fiscalYear"] = grant_pis["fiscalYear"].astype(int)
grant_pis.query(f"fiscalYear >= {START_FISCAL_YEAR}", inplace=True)
grant_pis["fiscalYear"] = grant_pis["fiscalYear"].astype(str)
grant_pis.sort_values("fiscalYear", ascending=False, inplace=True)
grant_pis.drop_duplicates("profileId", inplace=True)
grant_pis.rename(columns={"name": "grantPi"}, inplace=True)
print("Number of PIs:", grant_pis["grantPi"].nunique())
print(grant_pis.shape[0])
grant_pis.head()

Number of PIs: 99
99


,profileId,coreProjectNum,projectSerialNum,isContactPi,fiscalYear,grantPi,fullName,firstName,middleName,lastName
0,9436729,R01MD016526,MD016526,True,2023,Dillard DA,Denise A Dillard,Denise,A,Dillard
12,9942278,R01DK130067,DK130067,False,2023,Kotanko P,Peter Kotanko,Peter,,Kotanko
1,1877373,U01DC019578,DC019578,True,2023,Dalton PH,Pamela Helen Dalton,Pamela,Helen,Dalton
22,7086603,U01HL152410,HL152410,False,2023,Grant SA,SHEILA Ann GRANT,Sheila,Ann,Grant
21,1897028,U01HL152410,HL152410,True,2023,Fay WP,William P Fay,William,P,Fay


## Merge study investigators with grant investigators

In [9]:
investigators = utils.fuzzy_merge(projects[["dbgapAccession", "coreProjectNum", "studyInvestigator"]], grant_pis, left_fuzzy_on="studyInvestigator", right_fuzzy_on="grantPi", left_on="coreProjectNum", right_on="coreProjectNum", how="outer", threshold=0.9)
investigators = investigators[~((investigators["studyInvestigator"] == "") & (investigators["grantPi"] == ""))]

## Assign investigator roles and name

In [10]:
investigators["isPi"] = investigators["grantPi"] != ""
investigators["isDepositor"] = (investigators["studyInvestigator"] != "") & (investigators["grantPi"] == "")
investigators["name"] = investigators.apply(lambda x: x["studyInvestigator"] if x["grantPi"] == "" else x["grantPi"], axis=1)

## Cleanup

In [11]:
investigators.drop("match", axis=1, inplace=True)
investigators.drop("score", axis=1, inplace=True)
investigators.fillna("", inplace=True)
investigators

,dbgapAccession,coreProjectNum,studyInvestigator,profileId,projectSerialNum,isContactPi,fiscalYear,grantPi,fullName,firstName,middleName,lastName,isPi,isDepositor,name
0,phs002744.v1.p1,U01AA029345,Salaita K,8668731,AA029345,True,2022,Salaita KS,Khalid S Salaita,Khalid,S,Salaita,True,False,Salaita KS
1,phs002778.v1.p1,U18TR003793,Stott S,8956784,TR003793,True,2022,Stott SL,Shannon L Stott,Shannon,L,Stott,True,False,Stott SL
2,phs002604.v1.p1,U01DA053949,Uhlemann A,9733196,DA053949,True,2022,Uhlemann A,Anne-Catrin Uhlemann,Anne-Catrin,,Uhlemann,True,False,Uhlemann A
3,phs002583.v1.p1,U01HL150852,DeMauro EP,,,,,,,,,,False,True,DeMauro EP
4,phs002603.v1.p1,R33HD105594,Odom-John A,9198778,HD105594,True,2023,Odom John AR,Audrey Ragan Odom John,Audrey,Ragan,Odom John,True,False,Odom John AR
5,phs002642.v1.p1,R01DC016112,Travers S,1896926,DC016112,True,2021,Travers SP,Susan P Travers,Susan,P,Travers,True,False,Travers SP
6,phs002702.v1.p1,R44DE030842,Ly J,78421272,DE030842,True,2022,Ly J,Jeffrey Ly,Jeffrey,,Ly,True,False,Ly J
7,phs002685.v1.p1,R44DE030852,Yao X,77861758,DE030852,True,2022,Yao X,Xiaohu Yao,Xiaohu,,Yao,True,False,Yao X
8,phs003124.v1.p1,U18TR003780,Das S,10349485,TR003780,True,2022,Das S,Samarjit Das,Samarjit,,Das,True,False,Das S
9,phs002729.v1.p1,U01DA053899,Noble R,10129440,DA053899,True,2022,Noble RT,Rachel Todd Noble,Rachel,Todd,Noble,True,False,Noble RT


In [12]:
investigators.to_csv(os.path.join(DERIVED_DATA_PATH, "grant_investigators.csv"))

## Create a list of RADx Investigators
This list of investigators is used to filter publications related to RADx grants. Some RADx investigators are not the PI on a grant (depositors_only). They lead a subproject funded by the PI's grant. In this case, we want to remove the PIs  as their work is not related to RADx activities. This happens for example with Center grants that fund subprojects.

In [13]:
depositors_only = investigators[~investigators["isPi"] & investigators["isDepositor"]].copy()

In [14]:
depositors_only

,dbgapAccession,coreProjectNum,studyInvestigator,profileId,projectSerialNum,isContactPi,fiscalYear,grantPi,fullName,firstName,middleName,lastName,isPi,isDepositor,name
3,phs002583.v1.p1,U01HL150852,DeMauro EP,,,,,,,,,,False,True,DeMauro EP
16,phs002561.v1.p1,U54HL119145,Shafiee H,,,,,,,,,,False,True,Shafiee H
17,phs002602.v1.p1,U54HL119145,Unlu S,,,,,,,,,,False,True,Unlu S
18,phs002572.v1.p1,R42DE030832,Gordon T,,,,,,,,,,False,True,Gordon T
22,phs002924.v1.p1,U01HL152401,MacKenzie D,,,,,,,,,,False,True,MacKenzie D
33,phs002522.v1.p1,U01HL152410,Huang J,,,,,,,,,,False,True,Huang J
47,phs002542.v1.p1,U01DA053903,Keck JW,,,,,,,,,,False,True,Keck JW


In [15]:
grants = set(depositors_only["coreProjectNum"].unique())
radx_investigators = investigators[~investigators["coreProjectNum"].isin(grants)]
print(radx_investigators.shape[0])
radx_investigators = pd.concat([radx_investigators, depositors_only])
print(radx_investigators.shape[0])
radx_investigators

87
94


,dbgapAccession,coreProjectNum,studyInvestigator,profileId,projectSerialNum,isContactPi,fiscalYear,grantPi,fullName,firstName,middleName,lastName,isPi,isDepositor,name
0,phs002744.v1.p1,U01AA029345,Salaita K,8668731,AA029345,True,2022,Salaita KS,Khalid S Salaita,Khalid,S,Salaita,True,False,Salaita KS
1,phs002778.v1.p1,U18TR003793,Stott S,8956784,TR003793,True,2022,Stott SL,Shannon L Stott,Shannon,L,Stott,True,False,Stott SL
2,phs002604.v1.p1,U01DA053949,Uhlemann A,9733196,DA053949,True,2022,Uhlemann A,Anne-Catrin Uhlemann,Anne-Catrin,,Uhlemann,True,False,Uhlemann A
4,phs002603.v1.p1,R33HD105594,Odom-John A,9198778,HD105594,True,2023,Odom John AR,Audrey Ragan Odom John,Audrey,Ragan,Odom John,True,False,Odom John AR
5,phs002642.v1.p1,R01DC016112,Travers S,1896926,DC016112,True,2021,Travers SP,Susan P Travers,Susan,P,Travers,True,False,Travers SP
6,phs002702.v1.p1,R44DE030842,Ly J,78421272,DE030842,True,2022,Ly J,Jeffrey Ly,Jeffrey,,Ly,True,False,Ly J
7,phs002685.v1.p1,R44DE030852,Yao X,77861758,DE030852,True,2022,Yao X,Xiaohu Yao,Xiaohu,,Yao,True,False,Yao X
8,phs003124.v1.p1,U18TR003780,Das S,10349485,TR003780,True,2022,Das S,Samarjit Das,Samarjit,,Das,True,False,Das S
9,phs002729.v1.p1,U01DA053899,Noble R,10129440,DA053899,True,2022,Noble RT,Rachel Todd Noble,Rachel,Todd,Noble,True,False,Noble RT
10,phs002964.v1.p1,U01DC019579,Albers M,6625336,DC019579,True,2022,Albers MW,Mark W Albers,Mark,W,Albers,True,False,Albers MW


In [16]:
# TODO there should be 48 dbgapAccession numbers!
radx_investigators.query("dbgapAccession != ''").shape[0]

47

In [17]:
radx_investigators.to_csv(os.path.join(DERIVED_DATA_PATH, "radx_investigators.csv"), index=False)